In [1]:
# импорт библиотек
import cv2
import numpy as np
import pandas as pd
import pytesseract
import easyocr
import re

from PIL import Image

from IPython.display import display

## Функции

### Функция распознавания `img2table`

In [51]:
# функция для распознавания таблиц с изображения
# pytesseract для текста + EasyOCR для недостающих (digit=0)
# или + EasyOCR для цифр (digit=1)
def img2table(img_path, output_path, digit=0):
    # Считывание изображения
    img = Image.open(img_path)
    img = np.array(img)
    if img.ndim == 3 and img.shape[2] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Присвоение изображению порогового значения в виде двоичного изображения
    img_bin = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 21, 10)

    # Ширина ядра как 100-я часть общей ширины
    kernel_len = np.array(img).shape[1] // 100

    # Определение вертикального и горизонтального ядер
    ver_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_len))
    hor_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_len, 1))

    # Ядро размером 2x2
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))

    # Обнаружение вертикальных и горизонтальных линий
    image_1 = cv2.erode(img_bin, ver_kernel, iterations=3)
    vertical_lines = cv2.dilate(image_1, ver_kernel, iterations=3)

    image_2 = cv2.erode(img_bin, hor_kernel, iterations=3)
    horizontal_lines = cv2.dilate(image_2, hor_kernel, iterations=3)

    # Объединение горизонтальных и вертикальных линий в новом изображении
    img_vh = cv2.addWeighted(vertical_lines, 0.5, horizontal_lines, 0.5, 0.0)

    # Размывание и установление порогового значения
    img_vh = cv2.erode(~img_vh, kernel, iterations=2)
    _, img_vh = cv2.threshold(img_vh, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    # Обнаружение контуров
    contours, _ = cv2.findContours(img_vh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Преобразование tuple в list
    contours = list(contours)
    contours.reverse()

    # Подсчёт вертикальных линий (столбцов)
    num_vertical_lines = 1
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        
        if w > 20 and h > 20 and w < 500 and h < 500:
            num_vertical_lines = int(img.shape[1] / w)
            break
    
    texts = []
    # Итерация по каждому контуру (ячейке)
    for contour in contours:
        
        # Получение координат ограничивающего прямоугольника вокруг контура
        x, y, w, h = cv2.boundingRect(contour)
        if w > 20 and h > 20 and w < 500 and h < 500:
            cell_image = img[y:y + h, x:x + w]
        
            # Получение текста с ячейки
            text = pytesseract.image_to_string(cell_image, lang='rus')

            if digit == 0: # вариант "EasyOCR для недостающих"
                if text == '':
                    results = reader.readtext(cell_image)
                    if len(results) > 0:
                        text = results[0][1]
            else: # вариант "EasyOCR для цифр"
                if text == '' or re.search(r'\d', text):
                    results = reader.readtext(cell_image)
                    if len(results) > 0:
                        text = results[0][1]

            # Добавление текста в список
            texts.append(text.replace('|', '').replace('_', '').replace(',', '.'))
    
    # Разделение списка по колучиству столбцов
    texts = [texts[i:i+num_vertical_lines] for i in range(0, len(texts), num_vertical_lines)]
    
    # Создание датафрейма
    try:
        df = pd.DataFrame(data=texts[1:], columns=texts[0])
    except:
        df = pd.DataFrame(columns=texts)
    df.dropna(inplace=True)
    df = df.applymap(lambda x: x.replace('\n', ''))
    df = df.rename(columns=lambda x: re.sub(r'\s+', ' ', x.replace('\n', ' ')))
    
    # Сохранение
    df.to_csv(output_path, index=False)

    return df

### Функция сравнения результатов `compare_tables`

In [52]:
# функция сравнения результатов распознавания с реальными значениями
# df1 - датасет с реальными значениями
# df2 - датасет с результатами распознавания
def compare_tables(df1, df2):
    # считаем, что правильной является 1 таблица
    
    num_matching_cells = (df1 == df2).sum().sum()

    total_cells = df1.size
    
    accuracy = num_matching_cells/total_cells
    return num_matching_cells, total_cells, accuracy

## Основная программа

In [4]:
# активация pytesseract
pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract.exe'

In [5]:
# активация easyocr
reader = easyocr.Reader(['ru'])

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
c:\Users\Victor\anaconda3\lib\site-packages\torchvision\models\_utils.py:252: UserWarning: Accessing the model URLs via the internal dictionary of the module is deprecated since 0.13 and will be removed in 0.15. Please access them via the appropriate Weights Enum instead.
  warnings.warn(
c:\Users\Victor\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\Users\Victor\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [84]:
# назначение входного файла с изображением и файла csv с распознанной информацией

#path = 'C:/Users/Anastasia/Downloads/'
#img_path = path + '228963 .jpg'
img_path = 'tablica1.jpg'

#easy_dig_path = path + 'easy_dig.csv'
easy_dig_path = 'easy_dig.csv'

In [85]:
# вызов функции распознавания (возвращает датафрейм df)
df = img2table(img_path, easy_dig_path)

# отображение распознанной информации
print('Распознанная информация:')
display(df)

Распознанная информация:


,Дата,Вид дон-ва,Кол-во,Дата,Вид дон-ва,Кол-во,Дата,Вид дон-ва,Кол-во
0,,,,,,,,,
1,14.02.2006,кр/д (бв),420,15.07.2016,кр/д (бв),450,04.08.2018,кр/д (бв),450
2,11.06.2014,кр/д (бв),350,11.10.2016,кр/д (бв),450,26.12.2018,кр/д (бв),450
3,30.10.2014,кр/д (бв),450,21.12.2016,кр/д (бв),450,29.03.2019,кр/д (бв),450
4,13.08.2015,кр/д (бв),450,21.06.2017,кр/д (бв),450,11.10.2022,кр/д (бв),450
5,30.10.2015,кр/д (бв),450,,,,,,


In [86]:
# загрузка и отображение проверочной информации
df_test = pd.read_csv('tablica1.csv')
print('Проверочная информация:')
display(df_test)

Проверочная информация:


,0,1,2,3,4,5,6,7,8
0,Дата,Вид дон-ва,Количество,Дата,Вид дон-ва,Количество,Дата,Вид дон-ва,Количество
1,14.02.2006,кр/д (бв),420,15.07.2016,кр/д (бв),450,04.08.2018,кр/д (бв),450
2,11.06.2014,кр/д (бв),350,11.10.2016,кр/д (бв),450,26.12.2018,кр/д (бв),450
3,30.10.2014,кр/д (бв),450,21.12.2016,кр/д (бв),450,29.03.2019,кр/д (бв),450
4,13.08.2015,кр/д (бв),450,21.06.2017,кр/д (бв),450,11.10.2022,кр/д (бв),450
5,30.10.2015,кр/д (бв),450,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
# переименование столбцов в распознанном df, как в проверочном df_test
df.columns = [str(x) for x in list(range(9))]

# замена NaN в проверочном датасете
df_test.fillna('', inplace=True)

# удаление первых строк (пустой в df, с заголовками в df_test)
df = df.drop(labels=0, axis=0)
df_test = df_test.drop(labels=0, axis = 0)

# удаление пробелов в ячейках
df = df.applymap(lambda x: x.replace(' ', ''))
df_test = df_test.applymap(lambda x: x.replace(' ', ''))

# вызов функции сравнения результата распознавания
num_matching_cells, total_cells, accuracy = compare_tables(df_test, df)

# вывод результата сверки
print(f'Корректно распознанных ячеек: {accuracy*100:0.2f} %.')

Корректно распознанных ячеек: 100.00 %.
